# Net.exe Execution
Detects execution of Net.exe, whether suspicious or benign.

## Rule Content
```
- title: Net.exe Execution
  id: 183e7ea8-ac4b-4c23-9aec-b3dac4e401ac
  status: experimental
  description: Detects execution of Net.exe, whether suspicious or benign.
  references:
  - https://pentest.blog/windows-privilege-escalation-methods-for-pentesters/
  author: Michael Haag, Mark Woan (improvements)
  tags:
  - attack.s0039
  - attack.lateral_movement
  - attack.discovery
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\net.exe'
      - '*\net1.exe'
      CommandLine:
      - '* group*'
      - '* localgroup*'
      - '* user*'
      - '* view*'
      - '* share'
      - '* accounts*'
      - '* use*'
      - '* stop *'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Will need to be tuned. If using Splunk, I recommend | stats count by Computer,CommandLine
    following the search for easy hunting by computer/CommandLine.
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\net.exe OR *\\net1.exe) AND process_command_line.keyword:(*\ group* OR *\ localgroup* OR *\ user* OR *\ view* OR *\ share OR *\ accounts* OR *\ use* OR *\ stop\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()